In [74]:
import torch
import torchvision
from PIL import Image
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import numpy as np

In [81]:

def evaluate_gaussian(mu, sigma, x):
    coef_norm = 1 / (torch.sqrt(torch.tensor(2.0 * torch.pi * sigma ** 2)))
    return coef_norm * torch.exp(torch.tensor(-0.5*((mu - x)/sigma )** 2))
    

def binarize_image(image_tensor):
    image_tensor[image_tensor > 0.5] = 1
    image_tensor[image_tensor <= 0.5] = 0
    return image_tensor

def imshow(image_tensor):
    im = plt.imshow(image_tensor.numpy(), cmap = 'gray')
    plt.show()

#def test_model(input_torch, p_m_pix_val_given_k, p_t_tensor, num_classes = 10):
    #assumes that the input comes in a row
    #TODO IMPLEMENT
    #return (predicted_label, scores_classes)

#def test_model_batch(test_set, labels, p_m_pix_val_given_k, p_t_tensor):
    #TODO IMPLEMENT

def train_model(train_data_tensor, labels_training, num_classes = 10):
    _, labels_counts = torch.unique(labels_training, return_counts=True)
    p_t_tensor = labels_counts / labels_training.shape[0]
    #p_t_tensor 1 x 10
    
    #matched_values, counts = torch.unique(train_data_tensor[783,0:60], return_counts=True)
    #print("Values matched: %s\nCounts: %s" %(matched_values, counts))
    #result = counts/60
    #print("p_m_pix_val_given_k = ", result)
    first_tensor = True
    p_m_pix_val_given_k = None
    
    for img_index in range(0, num_classes * 60, 60):
        matched_values, counts = torch.unique(train_data_tensor[650,img_index:img_index+60], return_counts=True)
        print("Values matched: %s\nCounts: %s" %(matched_values, counts))
        
        if(matched_values[0] == 0):
            p_m = 1 - counts[0]/60
        else:
            p_m = counts[0]/60
        
        p_m_pix_val_given_k = p_m
        
        #if(first_tensor):
            #first_tensor = False
            #p_m_pix_val_given_k = p_m
        #else:
            #p_m_pix_val_given_k = torch.cat((p_m_pix_val_given_k, p_m), 1)
            
        print("p_m_pix_val_given_k = ", p_m_pix_val_given_k)
    
    #p_m_pix_val_given_k
    
    return (p_m_pix_val_given_k, p_t_tensor)

def load_dataset(path = "src_base/mnist_dataset/train"):
    #Open up the dataset
    dataset =  torchvision.datasets.ImageFolder(path)   
    #print(dataset)
    list_images = dataset.imgs
    #print(list_images)
    train_data_tensor  = None 
    train_data_tensor_gray = None
    labels_training = []
    first_tensor = True
    #list_images_training =  set(data_labeled.train_ds.x.items)
    #print(list_images)
    for i in range(len(list_images)):
        pair_path_label = list_images[i]
        image = Image.open(pair_path_label[0])        

        x_tensor = TF.to_tensor(image).squeeze() #Return tensor in range [0.0 - 1.0]
        x_tensor_gray = TF.pil_to_tensor(image).squeeze() #Return tensor in grayscale
        x_tensor_bin = binarize_image(x_tensor)
               
        #plt.figure()
        #plt.imshow(x_tensor_bin)
        x_tensor_bin_plain = x_tensor_bin.view(x_tensor_bin.shape[0] * x_tensor_bin.shape[1], -1)
        x_tensor_gray_plain = x_tensor_gray.view(x_tensor_gray.shape[0] * x_tensor_gray.shape[1], -1)
        #print("tensor ", x_tensor_bin_plain)
        #test dataset case        
        #if("train" in pair_path_label[0]):
        labels_training += [pair_path_label[1]]
        #print(pair_path_label)
        if(first_tensor):
            first_tensor = False
            train_data_tensor = x_tensor_bin_plain
            train_data_tensor_gray = x_tensor_gray_plain
        else:
            train_data_tensor = torch.cat((train_data_tensor, x_tensor_bin_plain), 1)  
            train_data_tensor_gray = torch.cat((train_data_tensor_gray, x_tensor_gray_plain), 1)
    return (train_data_tensor, train_data_tensor_gray, torch.tensor(labels_training))       

(train_data_tensor, train_data_tensor_gray, labels_training) = load_dataset()

print("train gray dimensions", train_data_tensor_gray.shape)
print("train dimensions ", train_data_tensor.shape)
print("train labels ", len(labels_training))

train_model(train_data_tensor, labels_training)

#train model by calculating the prior probabilities
#(p_m_pix_val_given_k, p_t_tensor) = train_model(train_data_tensor, labels_training)
#print("p_m_pix_val_given_k size ", p_m_pix_val_given_k.shape)
#(predicted_label, scores_classes) = test_model(train_data_tensor[:, 500], p_m_pix_val_given_k, p_t_tensor)
#print("predicted_label ", predicted_label)
#print("real label ", labels_training[500])
#acc = test_model_batch(train_data_tensor, labels_training, p_m_pix_val_given_k, p_t_tensor)
#print("Model accuracy ", acc)
    

               
    
    

train gray dimensions torch.Size([784, 600])
train dimensions  torch.Size([784, 600])
train labels  600
Values matched: tensor([0., 1.])
Counts: tensor([57,  3])
p_m_pix_val_given_k =  tensor(0.0500)
Values matched: tensor([0.])
Counts: tensor([60])
p_m_pix_val_given_k =  tensor(0.)
Values matched: tensor([0., 1.])
Counts: tensor([56,  4])
p_m_pix_val_given_k =  tensor(0.0667)
Values matched: tensor([0., 1.])
Counts: tensor([48, 12])
p_m_pix_val_given_k =  tensor(0.2000)
Values matched: tensor([0.])
Counts: tensor([60])
p_m_pix_val_given_k =  tensor(0.)
Values matched: tensor([0., 1.])
Counts: tensor([57,  3])
p_m_pix_val_given_k =  tensor(0.0500)
Values matched: tensor([0.])
Counts: tensor([60])
p_m_pix_val_given_k =  tensor(0.)
Values matched: tensor([0., 1.])
Counts: tensor([58,  2])
p_m_pix_val_given_k =  tensor(0.0333)
Values matched: tensor([0., 1.])
Counts: tensor([54,  6])
p_m_pix_val_given_k =  tensor(0.1000)
Values matched: tensor([0.])
Counts: tensor([60])
p_m_pix_val_given_

(tensor(0.),
 tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000]))

In [3]:
score_d_1_t_1 = evaluate_gaussian(mu = 20.25, sigma = 0.95, x = 7)
score_d_2_t_1 = evaluate_gaussian(mu = 59.75, sigma = 1.25, x = 17)
print("score_d_1_t_1 ", score_d_1_t_1)
print("score_d_2_t_1 ", score_d_2_t_1)
score_d_1_t_0 = evaluate_gaussian(mu = 5.5, sigma = 0.7, x = 7)
score_d_2_t_0 = evaluate_gaussian(mu = 17.5, sigma = 3.53, x = 17)
print("score_d_1_t_0 ", score_d_1_t_0)
print("score_d_2_t_0 ", score_d_2_t_0)

score_d_1_t_1  tensor(2.4102e-43)
score_d_2_t_1  tensor(0.)
score_d_1_t_0  tensor(0.0574)
score_d_2_t_0  tensor(0.1119)
